In [3]:
# coding=utf-8

import time
import csv
import pandas as pd
#S-boxes
SboxTable = [
[0xd6,0x90,0xe9,0xfe,0xcc,0xe1,0x3d,0xb7,0x16,0xb6,0x14,0xc2,0x28,0xfb,0x2c,0x05],
[0x2b,0x67,0x9a,0x76,0x2a,0xbe,0x04,0xc3,0xaa,0x44,0x13,0x26,0x49,0x86,0x06,0x99],
[0x9c,0x42,0x50,0xf4,0x91,0xef,0x98,0x7a,0x33,0x54,0x0b,0x43,0xed,0xcf,0xac,0x62],
[0xe4,0xb3,0x1c,0xa9,0xc9,0x08,0xe8,0x95,0x80,0xdf,0x94,0xfa,0x75,0x8f,0x3f,0xa6],
[0x47,0x07,0xa7,0xfc,0xf3,0x73,0x17,0xba,0x83,0x59,0x3c,0x19,0xe6,0x85,0x4f,0xa8],
[0x68,0x6b,0x81,0xb2,0x71,0x64,0xda,0x8b,0xf8,0xeb,0x0f,0x4b,0x70,0x56,0x9d,0x35],
[0x1e,0x24,0x0e,0x5e,0x63,0x58,0xd1,0xa2,0x25,0x22,0x7c,0x3b,0x01,0x21,0x78,0x87],
[0xd4,0x00,0x46,0x57,0x9f,0xd3,0x27,0x52,0x4c,0x36,0x02,0xe7,0xa0,0xc4,0xc8,0x9e],
[0xea,0xbf,0x8a,0xd2,0x40,0xc7,0x38,0xb5,0xa3,0xf7,0xf2,0xce,0xf9,0x61,0x15,0xa1],
[0xe0,0xae,0x5d,0xa4,0x9b,0x34,0x1a,0x55,0xad,0x93,0x32,0x30,0xf5,0x8c,0xb1,0xe3],
[0x1d,0xf6,0xe2,0x2e,0x82,0x66,0xca,0x60,0xc0,0x29,0x23,0xab,0x0d,0x53,0x4e,0x6f],
[0xd5,0xdb,0x37,0x45,0xde,0xfd,0x8e,0x2f,0x03,0xff,0x6a,0x72,0x6d,0x6c,0x5b,0x51],
[0x8d,0x1b,0xaf,0x92,0xbb,0xdd,0xbc,0x7f,0x11,0xd9,0x5c,0x41,0x1f,0x10,0x5a,0xd8],
[0x0a,0xc1,0x31,0x88,0xa5,0xcd,0x7b,0xbd,0x2d,0x74,0xd0,0x12,0xb8,0xe5,0xb4,0xb0],
[0x89,0x69,0x97,0x4a,0x0c,0x96,0x77,0x7e,0x65,0xb9,0xf1,0x09,0xc5,0x6e,0xc6,0x84],
[0x18,0xf0,0x7d,0xec,0x3a,0xdc,0x4d,0x20,0x79,0xee,0x5f,0x3e,0xd7,0xcb,0x39,0x48]
]

#Initial values of the key expansion algorithm
FK = [0xa3b1bac6, 0x56AA3350, 0x677d9197, 0xb27022dc]

#Parameters of the key expansion algorithm
CK =[
0x00070e15, 0x1c232a31, 0x383f464d, 0x545b6269,
0x70777e85, 0x8c939aa1, 0xa8afb6bd, 0xc4cbd2d9,
0xe0e7eef5, 0xfc030a11, 0x181f262d, 0x343b4249,
0x50575e65, 0x6c737a81, 0x888f969d, 0xa4abb2b9,
0xc0c7ced5, 0xdce3eaf1, 0xf8ff060d, 0x141b2229,
0x30373e45, 0x4c535a61, 0x686f767d, 0x848b9299,
0xa0a7aeb5, 0xbcc3cad1, 0xd8dfe6ed, 0xf4fb0209,
0x10171e25, 0x2c333a41, 0x484f565d, 0x646b7279
]
hex_status = []
#left circular rotation by n bits
def leftshift(a, n, size=32):
    n = n % size
    return ((a << n) | (a >> (size - n))) % (2**size)

#linear transformation of the key expansion algorithm
def PUT_ULONG_BE(b):
    b = sm4Sbox(b)
    return b ^ (leftshift(b, 13)) ^ (leftshift(b, 23))

#linear transformation of SM4
def GET_ULONG_BE(b):
    b = sm4Sbox(b)
    return b ^ (leftshift(b, 2)) ^ (leftshift(b, 10)) ^ (leftshift(b, 18)) ^ (leftshift(b, 24))

#Nonlinear tranformation (S-boxes) of SM4 and the key expansion algorithm
def sm4Sbox(a):
    b1 = SboxTable[(a & 0xf0000000) >> 28][(a & 0x0f000000) >> 24]
    b2 = SboxTable[(a & 0x00f00000) >> 20][(a & 0x000f0000) >> 16]
    b3 = SboxTable[(a & 0x0000f000) >> 12][(a & 0x00000f00) >>  8]
    b4 = SboxTable[(a & 0x000000f0) >>  4][(a & 0x0000000f) >>  0]
    return (b1 << 24) | (b2 << 16) | (b3 << 8) | (b4 << 0)

#key expansion algorithm
def generate_key(MK):
    K  = [0] *36
    rk = [0] *32

    K[0] = MK[0] ^ FK[0]
    K[1] = MK[1] ^ FK[1]
    K[2] = MK[2] ^ FK[2]
    K[3] = MK[3] ^ FK[3]
    #print hex(sm4Sbox(K[1] ^ K[2] ^ K[3] ^ CK[0]))

    for i in xrange(32):
        K[i+4] = K[i] ^ PUT_ULONG_BE(K[i+1] ^ K[i+2] ^ K[i+3] ^ CK[i])
        rk[i] = K[i+4]
        hex_status.append(['hex: '+'{:08x}'.format(rk[i])]) #round keys
        print "rk[" + str(i) + "] = " + hex(rk[i])
    return rk

#Encryption
def sm4_encrypt(message, key, method='cbc'):
    MK = key
    X  = message + [0]*32
    rk = generate_key(MK)

    for i in xrange(32):
        X[i+4] = X[i] ^ GET_ULONG_BE(X[i+1] ^ X[i+2] ^ X[i+3] ^ rk[i])
        tmp = X[i+1] ^ X[i+2] ^ X[i+3] ^ rk[i]
        hex_status[i].append('hex: '+'{:08x}'.format(tmp)) #inputs of S-boxes in each round
        hex_status[i].append('hex: '+'{:08x}'.format(sm4Sbox(tmp))) #outputs of S-boxes in each round
        hex_status[i].append('hex: '+'{:08x}'.format(GET_ULONG_BE(tmp))) #outpus of linear transformation in each round
        hex_status[i].append('hex: '+'{:08x}'.format(X[i+4])) #outputs of each round
        hex_status[i].append('hex: '+'{:08x}'.format(X[i+1]) + ' ' + '{:08x}'.format(X[i+2]) + ' ' + '{:08x}'.format(X[i+3]) + ' ' + '{:08x}'.format(X[i+4])) #message status after each round
        hex_status[i].append('bin: '+'{:032b}'.format(rk[i]))
        hex_status[i].append('bin: '+'{:032b}'.format(tmp))
        hex_status[i].append('bin: '+'{:08b}'.format(SboxTable[(tmp & 0xf0000000) >> 28][(tmp & 0x0f000000) >> 24])) #S1-Z1
        #print '{:08b}'.format(SboxTable[(tmp & 0xf0000000) >> 28][(tmp & 0x0f000000) >> 24])
        hex_status[i].append('bin: '+'{:08b}'.format(SboxTable[(tmp & 0x00f00000) >> 20][(tmp & 0x000f0000) >> 16])) #S2-Z2
        hex_status[i].append('bin: '+'{:08b}'.format(SboxTable[(tmp & 0x0000f000) >> 12][(tmp & 0x00000f00) >>  8])) #S3-Z3
        hex_status[i].append('bin: '+'{:08b}'.format(SboxTable[(tmp & 0x000000f0) >>  4][(tmp & 0x0000000f) >>  0])) #S4-Z4
        hex_status[i].append('bin: '+'{:032b}'.format(GET_ULONG_BE(tmp)))  #P1-P32
        hex_status[i].append('bin: '+'{:032b}'.format(X[i+4]))
        hex_status[i].append('bin: '+'{:032b}'.format(X[i+1])) #L01-L32
        hex_status[i].append('bin: '+'{:032b}'.format(X[i+2])) #L33-L64
        hex_status[i].append('bin: '+'{:032b}'.format(X[i+3])) #R01-R32
        hex_status[i].append('bin: '+'{:032b}'.format(X[i+4])) #R33-R64
        print "X[" + str(i+4) + "] = " + hex(X[i+4])
    hex_Y = [hex(X[35]), hex(X[34]), hex(X[33]), hex(X[32])]
    for j in range(len(hex_Y)):
        print "ciphertext[" + str(j) + "] = " + hex_Y[j]
    Y = [X[35], X[34], X[33], X[32]]
    return Y

#Decryption - same structure as Encryption except using the reverse of key
def sm4_decrypt(ciphertext, key, method='cbc'):
    MK = key
    X  = ciphertext + [0]*32
    rk = generate_key(MK)

    rk.reverse()
    
    for i in xrange(32):
        X[i+4] = X[i] ^ GET_ULONG_BE(X[i+1] ^ X[i+2] ^ X[i+3] ^ rk[i])

    hex_Y = [hex(X[35]), hex(X[34]), hex(X[33]), hex(X[32])]
    for j in range(len(hex_Y)):
        print "plaintext[" + str(j) + "] = " + hex_Y[j]
    Y = [X[35], X[34], X[33], X[32]]
    return Y

if __name__ == '__main__':
    time.clock()
    message = [0x01234567,0x89abcdef,0xfedcba98,0x76543210] #test vector
    key = [0x01234567,0x89abcdef,0xfedcba98,0x76543210] #test key
    
    ciphertext = sm4_encrypt(message, key)
        # message = map(lambda x:int(x[:-1], 16), ciphertext)
    
    #for i in range(len(ciphertext)):
        #print '%#x'%ciphertext[i]
    #print ciphertext
    #print time.clock()
    # ciphertext = [0x681edf34, 0xd206965e, 0x86b3e94f, 0x536e4246]

    # plaintext = sm4_decrypt(ciphertext, key)
    # print hex_status
    
    col_names = ['Round Keys(32bits)','Inputs of S-boxes(4*8bits)','Outputs of S-boxes(4*8bits)','Outpus of Linear Transformation(32bits)','Outputs of each round(32bits)','Message Status after each round(4*32bits)','Round Keys(bin)','Inputs of S-boxes(bin)','Outputs of 1st S-box(S1-Z1)','Outputs of 2nd S-box(S2-Z2)','Outputs of 3rd S-box(S3-Z3)','Outputs of 4th S-box(S4-Z4)','Outputs of Linear Transformation(P1-P32)','Outputs of each round(bin)','1st Message Block(L01-L32)','2nd Message Block(L33-L64)','3rd Message Block(R01-R32)','4th Message Block(R33-R64)']

    test = pd.DataFrame(columns = col_names, data = hex_status )
    test.to_csv('/Users/zc-zhai/Desktop/1.csv')
        
    # 10000 times 61.7811434009 s 
    # 10000 times 7.86056015746 s
    # 10000 times 5.46304156202 s



rk[0] = 0xf12186f9
rk[1] = 0x41662b61
rk[2] = 0x5a6ab19a
rk[3] = 0x7ba92077
rk[4] = 0x367360f4
rk[5] = 0x776a0c61
rk[6] = 0xb6bb89b3
rk[7] = 0x24763151
rk[8] = 0xa520307c
rk[9] = 0xb7584dbd
rk[10] = 0xc30753ed
rk[11] = 0x7ee55b57
rk[12] = 0x6988608c
rk[13] = 0x30d895b7
rk[14] = 0x44ba14af
rk[15] = 0x104495a1
rk[16] = 0xd120b428
rk[17] = 0x73b55fa3
rk[18] = 0xcc874966
rk[19] = 0x92244439
rk[20] = 0xe89e641f
rk[21] = 0x98ca015a
rk[22] = 0xc7159060
rk[23] = 0x99e1fd2e
rk[24] = 0xb79bd80c
rk[25] = 0x1d2115b0
rk[26] = 0xe228aeb
rk[27] = 0xf1780c81
rk[28] = 0x428d3654
rk[29] = 0x62293496
rk[30] = 0x1cf72e5
rk[31] = 0x9124a012
X[4] = 0x27fad345
X[5] = 0xa18b4cb2
X[6] = 0x11c1e22a
X[7] = 0xcc13e2ee
X[8] = 0xf87c5bd5
X[9] = 0x33220757
X[10] = 0x77f4c297
X[11] = 0x7a96f2eb
X[12] = 0x27dac07f
X[13] = 0x42dd0f19
X[14] = 0xb8a5da02
X[15] = 0x907127fa
X[16] = 0x8b952b83
X[17] = 0xd42b7c59
X[18] = 0x2ffc5831
X[19] = 0xf69e6888
X[20] = 0xaf2432c4
X[21] = 0xed1ec85e
X[22] = 0x55a3ba22
X[23] = 0x124b18a